In [33]:
############
#spark.catalog.clearCache()

### --------------------------------------------------

In [1]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .appName("Jupyter with Spark") \
    .master("spark://master:7077") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minio") \
    .config("spark.hadoop.fs.s3a.secret.key", "minio123") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.jars", "/opt/spark/jars/hadoop-aws-3.3.4.jar,/opt/spark/jars/aws-java-sdk-bundle-1.12.262.jar") \
    .getOrCreate()

In [3]:
spark

In [6]:
df = spark.read.option("header", "true").parquet("s3a://mybucket/taxi/input/NYC-Taxi.parquet")

In [8]:
df.count()

2964624

In [10]:
df.column()

AttributeError: 'DataFrame' object has no attribute 'column'

In [6]:
result = df.groupBy("passenger_count").count()
result.show()

+---------------+-------+
|passenger_count|  count|
+---------------+-------+
|              0|  31465|
|              7|      8|
|              6|  22353|
|              5|  33506|
|              1|2188739|
|              3|  91262|
|              8|     51|
|              2| 405103|
|              4|  51974|
|              9|      1|
|           NULL| 140162|
+---------------+-------+



In [15]:
spark.stop()